In [1]:
import os
import sys
import pandas as pd
import numpy as np
import glob

In [2]:
def extract_short_key_sudoku(row):
    big_key = row[gkey]
    return ('.'.join(big_key.split('/')[-2:]))

def extract_short_key_rc(row):
    big_key = row[gkey]
    return ('.'.join(big_key.split('/')[2:]))

def extract_short_key_kp(row):
    big_key = row[gkey]
    return ('.'.join(big_key.split('/')[3:]))


def add_grpby_columns_rc(tab):
    new_cols = tab[gkey].str.split('/', expand=True)
    new_cols = new_cols.iloc[:,2:]
    new_colnames = ['data','method','num_cons','seed','ver']
    new_cols.columns = new_colnames
    tab = pd.concat([tab,new_cols], axis=1)
    tab[skey] = tab.apply(extract_short_key_rc, axis=1)
    return tab

def add_grpby_columns_kp(tab):
    #['runs', 'neurips', 'kp', 'ilploss', 'kp4', 'vggasbb_splinit', 'version_0']
    new_cols = tab[gkey].str.split('/', expand=True)
    new_cols = new_cols.iloc[:,3:]
    new_colnames = ['method','data','spl','ver']
    new_cols.columns = new_colnames
    tab = pd.concat([tab,new_cols], axis=1)
    tab[skey] = tab.apply(extract_short_key_kp, axis=1)
    return tab


exp_dir_list_sudoku = ["runs/neurips/sudoku/symbolic"]



grpby_keys_rc = ['data','method','num_cons','ver']
grp_over_rc = ['seed']
#exp_dir_list_rc = ["runs/neurips/binary_random",'runs/neurips/dense_random']
exp_dir_list_rc = ["runs/neurips/binary_random",
                   "runs/neurips/binary_random_50pct",
                   'runs/neurips/dense_random',
                   'runs/neurips/dense_random_50pct']
#exp_dir_list_rc = ["runs/neurips/binary_random_50pct"]
es_col_rc = 'val/acc/idv'
expected_count_rc = 10


expected_count_kp = 1
exp_dir_list_kp = ["runs/neurips/kp/"]
es_col_kp = 'val/acc/idv'
grpby_keys_kp = ['method','data','spl','ver']
grp_over_kp = ['data']


json_filename = 'training_stats.json'




In [ ]:
#tab.iloc[0]

In [3]:
test_only_list = [0,1]
version_list =  ['version_0', 'version_1']
#version_list =  ['version_0']
event_list = ['test_epoch_end']



## Input: 

In [4]:
dataset= 'kp'


In [5]:
pwd

'/home/yatin/softlinks/hpchome/phd/comboptnet/neurips_sudoku/misc/notebooks'

In [6]:
pwd

'/home/yatin/softlinks/hpchome/phd/comboptnet/neurips_sudoku/misc/notebooks'

In [7]:
base_dir ="../.."
os.chdir(base_dir)

In [92]:
gkey = 'log_dir'

total_train_time = 'total_time/train'
total_val_time = 'total_time/val'
epoch_val_time = 'epoch_time/val'
epoch_test_time  = 'epoch_time/test'
net_train_time = 'net_time/train'
val_timeout = 'val/status/time_limit'
val_optimal = 'val/status/optimal'
test_timeout = 'test/status/time_limit'
test_optimal = 'test/status/optimal'
val_acc = 'val/acc/all'
test_acc = 'test/acc/all'
event = 'event'
#derived columns:
val_count = 'val_count'
test_count = 'test_count'
skey = 'skey'
val_acc_num = 'val/acc/num'
test_acc_num = 'test/acc/num'
test_acc_pre = 'test/acc/pre'
relevant_cols = [skey,event,net_train_time,total_train_time,total_val_time,epoch_test_time,
                 val_timeout, test_timeout, val_count, test_count, 
                 val_acc, test_acc]

time_cols = [net_train_time,total_train_time,total_val_time,epoch_test_time]

In [93]:
if dataset== 'rc':
    add_grpby_columns = add_grpby_columns_rc
    grpby_keys = grpby_keys_rc
    exp_dir_list = exp_dir_list_rc
    expected_count = expected_count_rc
    grp_over = grp_over_rc
    es_col = es_col_rc
    stat_cols = [net_train_time,total_train_time,total_val_time,epoch_test_time,val_acc,test_acc]
    val_metric = val_acc
    test_metric = test_acc

elif dataset=='kp':
    
    exp_dir_list = exp_dir_list_kp
    add_grpby_columns = add_grpby_columns_kp
    expected_count = expected_count_kp
    es_col = es_col_kp
    grpby_keys = grpby_keys_kp
    grp_over = grp_over_kp
    stat_cols = [net_train_time,total_train_time,total_val_time,epoch_test_time,val_acc,test_acc, val_acc_num, test_acc_num, test_acc_pre]
    val_metric = val_acc_num
    test_metric = test_acc_num
    
    
else:
    raise

In [10]:
dataset

'kp'

In [11]:
exp_dir_list

['runs/neurips/kp/']

In [12]:
ls runs/neurips/kp/

comboptnet/  ilploss/  neural/


In [13]:
all_files = []
for exp_dir in exp_dir_list:
    for dirpath, dirnames, filenames in os.walk(exp_dir, topdown= True):
        #print(dirpath, dirnames,filenames)
        to_skip = []
        for temp_ind, this_exp_dir in enumerate(dirnames):
            log_file = os.path.join(dirpath, this_exp_dir, json_filename)
            #print(log_file)
            if os.path.exists(log_file):
                all_files.append(log_file)
                to_skip.append(temp_ind)
            #
            #prune search
        #print("Skipping search in:",[dirnames[x] for x in to_skip])
        for temp_i in range( len(to_skip) - 1, -1, -1):
            del dirnames[to_skip[temp_i]]
        
    

In [14]:
all_files

['runs/neurips/kp/ilploss/kp5/vggasbb_pretrain/version_0/training_stats.json',
 'runs/neurips/kp/ilploss/kp4/vggasbb_splinit/version_0/training_stats.json',
 'runs/neurips/kp/ilploss/kp6/vggasbb_pretrain/version_0/training_stats.json',
 'runs/neurips/kp/ilploss/kp7/vggasbb_pretrain/version_0/training_stats.json',
 'runs/neurips/kp/neural/kp5/vggasbb_pretrain/version_0/training_stats.json',
 'runs/neurips/kp/neural/kp4/vggasbb/version_0/training_stats.json',
 'runs/neurips/kp/neural/kp6/vggasbb_pretrain/version_0/training_stats.json',
 'runs/neurips/kp/neural/kp7/vggasbb_pretrain/version_0/training_stats.json',
 'runs/neurips/kp/comboptnet/kp5/ext_inituni_pretrain/version_0/training_stats.json',
 'runs/neurips/kp/comboptnet/kp4/ext_inituni/version_0/training_stats.json',
 'runs/neurips/kp/comboptnet/kp6/ext_inituni/version_0/training_stats.json',
 'runs/neurips/kp/comboptnet/kp7/ext_inituni_pretrain/version_0/training_stats.json']

In [15]:
print("Fetched all desired file names.. now reading them..", len(all_files))

Fetched all desired file names.. now reading them.. 12


In [17]:
all_tables = []
for this_file in all_files:
    this_table = pd.read_json(this_file, orient = 'records',lines = True)
    this_table['logdir'] = os.path.dirname(this_file)
    all_tables.append(this_table)

tab = pd.concat(all_tables)

In [18]:
len(all_tables)

12

In [19]:
tab.shape

(354, 33)

In [20]:
#tab.iloc[0]['log_dir'].split('/')

In [21]:
tab = add_grpby_columns(tab)

In [22]:
tab.reset_index(inplace=True,drop=True)

In [23]:
tab[net_train_time] = tab[total_train_time] - tab[total_val_time]

In [24]:
#extract short key
for col in [val_timeout,test_timeout, val_optimal, test_optimal]:
    if col not in tab.columns:
        tab[col] = 0
    

tab[val_count] = tab[val_optimal] + tab[val_timeout]

tab[test_count] = tab[test_optimal] + tab[test_timeout]

tab[time_cols] = tab[time_cols]/60.0

In [25]:
es_col

'val/acc/idv'

In [26]:
time_tab = tab.iloc[tab.groupby(skey)[es_col].idxmax().dropna()]

In [27]:
count_time = time_tab.groupby(grpby_keys)[grp_over[0]].count().to_frame()

In [70]:
time_stats = time_tab.groupby(grpby_keys)[stat_cols+['epoch']].agg(['mean','std']).round(4)

In [37]:
time_stats_print = time_stats[[net_train_time]].transpose()

In [77]:
time_stats

net_time/train      \
                                                         mean std   
method     data spl                  ver                            
comboptnet kp4  ext_inituni          version_0        74.2973 NaN   
           kp5  ext_inituni_pretrain version_0       112.0214 NaN   
           kp6  ext_inituni          version_0       307.9001 NaN   
           kp7  ext_inituni_pretrain version_0       286.6366 NaN   
ilploss    kp4  vggasbb_splinit      version_0       127.3910 NaN   
           kp5  vggasbb_pretrain     version_0        85.0659 NaN   
           kp6  vggasbb_pretrain     version_0        83.7378 NaN   
           kp7  vggasbb_pretrain     version_0        91.1279 NaN   
neural     kp4  vggasbb              version_0       178.1535 NaN   
           kp5  vggasbb_pretrain     version_0        59.9754 NaN   
           kp6  vggasbb_pretrain     version_0         7.2536 NaN   
           kp7  vggasbb_pretrain     version_0        51.6972 NaN   

                                               total_time/train      \
                                                           mean std   
method     data spl                  ver                              
comboptnet kp4  ext_inituni          version_0          79.0693 NaN   
           kp5  ext_inituni_pretrain version_0         121.5344 NaN   
           kp6  ext_inituni          version_0         359.8466 NaN   
           kp7  ext_inituni_pretrain version_0         358.4318 NaN   
ilploss    kp4  vggasbb_splinit      version_0         137.0813 NaN   
           kp5  vggasbb_pretrain     version_0          92.3689 NaN   
           kp6  vggasbb_pretrain     version_0          93.4771 NaN   
           kp7  vggasbb_pretrain     version_0         107.4383 NaN   
neural     kp4  vggasbb              version_0         192.2940 NaN   
           kp5  vggasbb_pretrain     version_0          65.1538 NaN   
           kp6  vggasbb_pretrain     version_0           8.1538 NaN   
           kp7  vggasbb_pretrain     version_0          56.5241 NaN   

                                               total_time/val      \
                                                         mean std   
method     data spl                  ver                            
comboptnet kp4  ext_inituni          version_0         4.7720 NaN   
           kp5  ext_inituni_pretrain version_0         9.5130 NaN   
           kp6  ext_inituni          version_0        51.9465 NaN   
           kp7  ext_inituni_pretrain version_0        71.7951 NaN   
ilploss    kp4  vggasbb_splinit      version_0         9.6904 NaN   
           kp5  vggasbb_pretrain     version_0         7.3029 NaN   
           kp6  vggasbb_pretrain     version_0         9.7393 NaN   
           kp7  vggasbb_pretrain     version_0        16.3105 NaN   
neural     kp4  vggasbb              version_0        14.1405 NaN   
           kp5  vggasbb_pretrain     version_0         5.1784 NaN   
           kp6  vggasbb_pretrain     version_0         0.9003 NaN   
           kp7  vggasbb_pretrain     version_0         4.8269 NaN   

                                               epoch_time/test      \
                                                          mean std   
method     data spl                  ver                             
comboptnet kp4  ext_inituni          version_0             NaN NaN   
           kp5  ext_inituni_pretrain version_0             NaN NaN   
           kp6  ext_inituni          version_0             NaN NaN   
           kp7  ext_inituni_pretrain version_0             NaN NaN   
ilploss    kp4  vggasbb_splinit      version_0             NaN NaN   
           kp5  vggasbb_pretrain     version_0             NaN NaN   
           kp6  vggasbb_pretrain     version_0             NaN NaN   
           kp7  vggasbb_pretrain     version_0             NaN NaN   
neural     kp4  vggasbb              version_0             NaN NaN   
           kp5  vggasbb_pretrain     version_0             NaN NaN   

In [86]:
pd.pivot_table(time_stats, index='method', columns=['data']).round(2)[[net_train_time, 'epoch']]

net_time/train                         epoch                  
                     mean                          mean                  
data                  kp4     kp5     kp6     kp7   kp4   kp5   kp6   kp7
method                                                                   
comboptnet          74.30  112.02  307.90  286.64  25.0  31.0  55.0  45.0
ilploss            127.39   85.07   83.74   91.13  55.0  33.0  31.0  31.0
neural             178.15   59.98    7.25   51.70  81.0  25.0   3.0  19.0

spl                                          \
                                                                      
data                    kp4                   kp5               kp6   
method                                                                
comboptnet      ext_inituni  ext_inituni_pretrain       ext_inituni   
ilploss     vggasbb_splinit      vggasbb_pretrain  vggasbb_pretrain   
neural              vggasbb      vggasbb_pretrain  vggasbb_pretrain   

                                        ver                                   \
                                                                               
data                         kp7        kp4        kp5        kp6        kp7   
method                                                                         
comboptnet  ext_inituni_pretrain  version_0  version_0  version_0  version_0   
ilploss         vggasbb_pretrain  version_0  version_0  version_0  version_0   
neural          vggasbb_pretrain  version_0  version_0  version_0  version_0   

           net_time/train            ... test/acc/num     epoch              \
                     mean            ...          std      mean               
data                  kp4       kp5  ...          kp6 kp7   kp4   kp5   kp6   
method                               ...                                      
comboptnet        74.2973  112.0214  ...          NaN NaN  25.0  31.0  55.0   
ilploss          127.3910   85.0659  ...          NaN NaN  55.0  33.0  31.0   
neural           178.1535   59.9754  ...          NaN NaN  81.0  25.0   3.0   

                                  
                 std              
data         kp7 kp4 kp5 kp6 kp7  
method                            
comboptnet  45.0 NaN NaN NaN NaN  
ilploss     31.0 NaN NaN NaN NaN  
neural      19.0 NaN NaN NaN NaN  

[3 rows x 80 columns]

In [39]:
#extract the relevant rows

In [40]:
event_list, test_only_list, version_list

(['test_epoch_end'], [0, 1], ['version_0', 'version_1'])

In [41]:
tab[event].unique()

array(['val_epoch_end', 'test_epoch_end'], dtype=object)

In [42]:
tab['ver'].unique()

array(['version_0'], dtype=object)

In [43]:
test_tab = tab[(tab[event].isin(event_list)) & (tab['test_only'].isin(test_only_list)) & (tab['ver'].isin(version_list))]

In [44]:
count_bf = test_tab.groupby(grpby_keys)[grp_over[0]].count().to_frame()

In [45]:
count_bf.shape

(12, 1)

In [46]:
count_bf

data
method     data spl                  ver            
comboptnet kp4  ext_inituni          version_0     1
           kp5  ext_inituni_pretrain version_0     1
           kp6  ext_inituni          version_0     1
           kp7  ext_inituni_pretrain version_0     1
ilploss    kp4  vggasbb_splinit      version_0     1
           kp5  vggasbb_pretrain     version_0     1
           kp6  vggasbb_pretrain     version_0     1
           kp7  vggasbb_pretrain     version_0     1
neural     kp4  vggasbb              version_0     1
           kp5  vggasbb_pretrain     version_0     1
           kp6  vggasbb_pretrain     version_0     1
           kp7  vggasbb_pretrain     version_0     1

In [47]:
if ((count_bf != expected_count).any().any()):
    print('Some exp. ran more than once. Please check. Picking the last one ...')
    

In [48]:
test_tab = test_tab.groupby(grpby_keys+grp_over).tail(1)

In [49]:
count = test_tab.groupby(grpby_keys)[grp_over[0]].count().to_frame()

In [50]:
assert (count == expected_count).all().all()

In [52]:
stat_cols

['net_time/train',
 'total_time/train',
 'total_time/val',
 'epoch_time/test',
 'val/acc/all',
 'test/acc/all',
 'val/acc/num',
 'test/acc/num']

In [94]:
all_stats = test_tab.groupby(grpby_keys)[stat_cols].agg(['mean','std'])

In [95]:
stats = all_stats.transpose().round(4)

In [55]:
test_stats = all_stats[[test_metric]].transpose().round(4)

In [88]:
pd.pivot_table(time_stats, index='method', columns=['data']).round(2)[[net_train_time,total_train_time, 'epoch']]

net_time/train                         total_time/train          \
                     mean                                     mean           
data                  kp4     kp5     kp6     kp7              kp4     kp5   
method                                                                       
comboptnet          74.30  112.02  307.90  286.64            79.07  121.53   
ilploss            127.39   85.07   83.74   91.13           137.08   92.37   
neural             178.15   59.98    7.25   51.70           192.29   65.15   

                           epoch                    
                            mean                    
data           kp6     kp7   kp4   kp5   kp6   kp7  
method                                              
comboptnet  359.85  358.43  25.0  31.0  55.0  45.0  
ilploss      93.48  107.44  55.0  33.0  31.0  31.0  
neural        8.15   56.52  81.0  25.0   3.0  19.0

In [100]:
100*pd.pivot_table(all_stats, index='method', columns=['data']).round(4)[[test_metric,test_acc_pre]]

test/acc/num                      net_time/train            \
                   mean                                mean             
data                kp4    kp5    kp6    kp7            kp4       kp5   
method                                                                  
comboptnet        84.27  80.32  79.24  79.35       13665.29  19323.72   
ilploss           82.28  78.43  76.97  73.23       17465.74  13754.05   
neural            83.10  79.73  77.86  75.79       21962.40  10918.84   

                               test/acc/pre                       
                                       mean                       
data             kp6       kp7          kp4    kp5    kp6    kp7  
method                                                            
comboptnet  42343.59  41776.37        68.00  69.66  67.94  72.76  
ilploss     13943.50  15390.62        53.49  60.13  68.81  67.44  
neural       6103.04  10872.85        82.00  78.47  75.72  74.10

In [104]:
pd.pivot_table(all_stats, index='method', columns=['data']).round(4)[[net_train_time]]

net_time/train                              
                     mean                              
data                  kp4       kp5       kp6       kp7
method                                                 
comboptnet       136.6529  193.2372  423.4359  417.7637
ilploss          174.6574  137.5405  139.4350  153.9062
neural           219.6240  109.1884   61.0304  108.7285

In [ ]:
result = pd.concat([count.transpose(),stats,time_stats])

In [ ]:
test_result = pd.concat([count.transpose().droplevel('ver',axis=1),
                         test_stats.droplevel('ver',axis=1),
                         time_stats.droplevel('ver',axis=1)],axis=0)

In [ ]:
"""
test_result = pd.concat([count.transpose(),
                         test_stats,
                         time_stats],axis=0)
                         
"""


In [ ]:
grpby_keys

In [ ]:
test_result.transpose().groupby(grpby_keys[:-1]).tail(n=1).transpose()

In [ ]:
test_result.transpose()

In [ ]:
time_stats.droplevel('ver',axis=1)

In [ ]:
test_result